In [ ]:
pip install --upgrade torch torchvision

In [ ]:
# https://drive.google.com/file/d/1S7upK1OsxQoLfOLDTR-Px3XJFGuZKn3B/view?usp=sharing
!gdown https://drive.google.com/uc?id=1S7upK1OsxQoLfOLDTR-Px3XJFGuZKn3B

In [ ]:
!rm -r TLDataset/ALL/PKG-C-NMC2019/train/.ipynb_checkpoints
!rm -r TLDataset/ALL/PKG-C-NMC2019/train/.DS_STORE

In [ ]:
from torchvision.models import ConvNeXt_Base_Weights

In [2]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [ ]:
pip install --upgrade efficientnet-pytorch

In [ ]:
!pip install torch-summary

In [ ]:
!pip install summary

# Training

In [6]:
# start training
!python train-mnv2.py --model_name mobilenetv2_100 --data_dir TLDataset/ALL/PKG-C-NMC2019 --num_classes 2 --num_epochs 50 --batch_size 32 --learning_rate 0.001 --checkpoint_path model_checkpoints/ALL/mnv2/

Using device: cuda:0
Training Data: 7845 samples
Validation Data: 2347 samples
Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 32, 112, 112]        864
├─BatchNormAct2d: 1-2                         [-1, 32, 112, 112]        --
|    └─Identity: 2-1                          [-1, 32, 112, 112]        --
|    └─ReLU6: 2-2                             [-1, 32, 112, 112]        --
├─Sequential: 1-3                             [-1, 320, 7, 7]           --
|    └─Sequential: 2-3                        [-1, 16, 112, 112]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 16, 112, 112]        896
|    └─Sequential: 2-4                        [-1, 24, 56, 56]          --
|    |    └─InvertedResidual: 3-2             [-1, 24, 56, 56]          5,136
|    |    └─InvertedResidual: 3-3             [-1, 24, 56, 56]          8,832
|    └─Sequential: 2-5                        [-1, 32, 28, 28]          --
|    |  

# Evaluation 

ALL Only Evaluation

In [4]:
!python evaluate-mnv2.py --model_name mobilenetv2_100 --data_dir /home/sagemaker-user/ALL_Unseen/ --model_path /home/sagemaker-user/model_checkpoints/ALL/mnv2/mobilenetv2_100_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|████████████████████| 15/15 [00:01<00:00, 10.80it/s, acc=0.953]

Evaluation Results:
Precision: 0.9561
Recall: 0.9351
F1: 0.9447
Accuracy: 0.9531
Balanced accuracy: 0.9351


# Inference

In [ ]:
!python inference.py --model_name resnet50 --image_path TLDataset/ALL/PKG-C-NMC2019/C-NMC_test_prelim_phase_data --model_path model_checkpoints/ALL/resnet50_best_epoch_49.pth --num_classes 2 --image_save results_C-NMC_test_prelim_phase_data

In [ ]:
!python inference.py --model_name resnet50 --image_path TLDataset/ALL/PKG-C-NMC2019/C-NMC_test_final_phase_data --model_path model_checkpoints/ALL/resnet50_best_epoch_49.pth --num_classes 2 --image_save results_C-NMC_test_final_phase_data

# Transfer learning 

Here the trained model on ALL dataset is finetuned for AML dataset, by loading the ALL model weights as pretrained model.

In [9]:
!python train-mnv2.py --model_name mobilenetv2_100 --data_dir AMLDataset/AML --num_classes 2 --num_epochs 50 --batch_size 32 --learning_rate 0.001 --checkpoint_path model_checkpoints/AML/mnv2 --pretrained_path model_checkpoints/ALL/mnv2/mobilenetv2_100_best.pth

Using device: cuda:0
Training Data: 12207 samples
Validation Data: 5230 samples
Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 32, 112, 112]        864
├─BatchNormAct2d: 1-2                         [-1, 32, 112, 112]        --
|    └─Identity: 2-1                          [-1, 32, 112, 112]        --
|    └─ReLU6: 2-2                             [-1, 32, 112, 112]        --
├─Sequential: 1-3                             [-1, 320, 7, 7]           --
|    └─Sequential: 2-3                        [-1, 16, 112, 112]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 16, 112, 112]        896
|    └─Sequential: 2-4                        [-1, 24, 56, 56]          --
|    |    └─InvertedResidual: 3-2             [-1, 24, 56, 56]          5,136
|    |    └─InvertedResidual: 3-3             [-1, 24, 56, 56]          8,832
|    └─Sequential: 2-5                        [-1, 32, 28, 28]          --
|    | 

# Notes

Inference on unseen AML Images

In [ ]:
# TL model with unseen ALL images again to see if TL helped

In [10]:
!python evaluate-mnv2.py --model_name mobilenetv2_100 --data_dir ALL_Unseen/ --model_path model_checkpoints/AML/mnv2/mobilenetv2_100_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|████████████████████| 15/15 [00:00<00:00, 18.00it/s, acc=0.322]

Evaluation Results:
Precision: 0.6595
Recall: 0.5031
F1: 0.2481
Accuracy: 0.3220
Balanced accuracy: 0.5031


In [ ]:
# TL model with unseen AML images

In [2]:
!python evaluate-mnv2.py --model_name mobilenetv2_100 --data_dir /home/sagemaker-user/AML_Unseen/ --model_path /home/sagemaker-user/model_checkpoints/AML/mnv2/mobilenetv2_100_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|████████████████████| 29/29 [00:03<00:00,  7.76it/s, acc=0.976]

Evaluation Results:
Precision: 0.9604
Recall: 0.9642
F1: 0.9623
Accuracy: 0.9760
Balanced accuracy: 0.9642


In [ ]:
# Training AML Only without TF weights to Compare with TF Model

In [ ]:
!pip install seaborn

In [12]:
# start training AML Model
!python train-mnv2.py --model_name mobilenetv2_100 --data_dir AMLDataset/AML/ --num_classes 2 --num_epochs 50 --batch_size 32 --learning_rate 0.001 --checkpoint_path model_checkpoints/AML_Only/mnv2/

Using device: cuda:0
Training Data: 12207 samples
Validation Data: 5230 samples
Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 32, 112, 112]        864
├─BatchNormAct2d: 1-2                         [-1, 32, 112, 112]        --
|    └─Identity: 2-1                          [-1, 32, 112, 112]        --
|    └─ReLU6: 2-2                             [-1, 32, 112, 112]        --
├─Sequential: 1-3                             [-1, 320, 7, 7]           --
|    └─Sequential: 2-3                        [-1, 16, 112, 112]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 16, 112, 112]        896
|    └─Sequential: 2-4                        [-1, 24, 56, 56]          --
|    |    └─InvertedResidual: 3-2             [-1, 24, 56, 56]          5,136
|    |    └─InvertedResidual: 3-3             [-1, 24, 56, 56]          8,832
|    └─Sequential: 2-5                        [-1, 32, 28, 28]          --
|    | 

In [ ]:
# Test AML_Only model on AML Unseen Data

In [3]:
!python evaluate-mnv2.py --model_name mobilenetv2_100 --data_dir /home/sagemaker-user/AML_Unseen/ --model_path /home/sagemaker-user/model_checkpoints/AML_Only/mnv2/mobilenetv2_100_best.pth --num_classes 2 --batch_size 32

Evaluating: 100%|████████████████████| 29/29 [00:01<00:00, 24.64it/s, acc=0.969]

Evaluation Results:
Precision: 0.9470
Recall: 0.9581
F1: 0.9524
Accuracy: 0.9695
Balanced accuracy: 0.9581


In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
# Best AML model weights to TF on ALL data

In [ ]:
!python train-mobilenetv2.py --model_name mobilenetv2 --data_dir TLDataset/ALL/PKG-C-NMC2019 --num_classes 2 --num_epochs 150 --batch_size 64 --learning_rate 0.01 --momentum 0.9 --checkpoint_path model_checkpoints/AML_Pretrained/mobilenetv2 --pretrained_path model_checkpoints/AML_Only/mobilenetv2/mobilenetv2_best_epoch_104.pth

In [ ]:
# TF Model Validation on Unseen ALL

In [ ]:
!python evaluate-mobilenetv2.py --model_name mobilenetv2 --data_dir ALL_Unseen/ --model_path model_checkpoints/AML_Pretrained/mobilenetv2/mobilenetv2_best_epoch_100.pth --num_classes 2 --batch_size 32

In [ ]:
# TF Model Validation on Unseen AML

In [ ]:
!python evaluate.py --model_name resnet50 --data_dir AML_Unseen/ --model_path model_checkpoints/ALL_Pretrained/resnet50_best_epoch_76.pth --num_classes 2 --batch_size 32